# Protein Secondary Structure Prediction - Tensorflow

Using 75/130 proteins from the  Rost-Sander database that share than 25% identity. 

Links:

http://antheprot-pbil.ibcp.fr/Rost.html

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.17.1418&rep=rep1&type=pdf

The database protein amino acid sequences for 130 protein, and their secondary structure.  Using only the amino-acid sequence, I train a Deep neural network  to predict the secondary structure of the  individual amino acid in the peptide chain.  A sliding window of 17 amino acid  as the input, and predict secondary structure of the amino acid in the middle. The window size of 17 is selected because of the correlation found between the secondary structure and eight amino acids on either sides of the central amino acid of interest.

## IMPORT

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import itertools

# The data was originally saved as .mat from matlab.
from scipy.io import loadmat
from scipy.sparse import kron
from scipy.linalg import hankel
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,classification_report

### Create a dictionary to convert the amino acid sequences to numeric sequences.

In [2]:
aa_dict = {1:'A',2:'R',3:'N',4:'D',5:'C',6:'Q',7:'E',8:'G',9:'H',10:'I',
          11:'L',12:'K',13:'M',14:'F',15:'P',16:'S',17:'T',18:'W',19:'Y',
          20:'V',21:'B',22:'Z',23:'X',24:'*',25:'-',26:'?'}


#### Load and display the data

In [3]:
data = loadmat('/Users/Jigar/hubiC/Documents/Matlab/Ross_Saunder.mat',squeeze_me=True)  

In [4]:
N = data['allSeq']['Sequence'].shape[0]  
print ("Total no. of proteins in the dataset = {}".format(N))

#Let's look at the identity of protein_no = 6
protein_no = 6
protein_id = data['allSeq']['Header'][protein_no]
seq1_n = data['allSeq']['Sequence'][protein_no]
str1 = data['allSeq']['Structure'][protein_no]

print ('Protein id : ',protein_id)
print ('Protein Sequence : ',''.join([aa_dict.get(x) for x in seq1_n]))
print ('Protein Structure : ', str1)

Total no. of proteins in the dataset = 75
Protein id :  1CSE-ICOMPLEX(SERINEPROTEINASE-INHIBITOR)03-JU
Protein Sequence :  KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNHVPHVG
Protein Structure :  CCCHHHCCCCHHHHHHHHHHHCCCCEEEEEECCCCEECCCCCCEEEEEEECCCCEECCCCEEC


In [5]:
W = 17
win = hankel(seq1_n[:W],seq1_n[W-1:])


Each amino acid is encoded as 1-D binary array of size 20. In this array, the element corresponding to the amino acid is set to 1 and all the other elements are set to 0. Since each input window consists of 17 amino acids, the actual input is 17 x 20 = 340 features.

Similarly, the secondary structures C,E and H are convert to one hot vector.

In [60]:
#This part of code converts all the protein sequences to desired input format. Each amino acid in the protein 
#is represented as a 17 amino acid sliding window, with each amino acid being encoded as a 20 element 1-D array.
#The output for each amino acid is convert to a one hot vector of size 3.

X_temp = []
y_temp = []
for i in range(N):
    protein_no = i
    protein_id = data['allSeq']['Header'][protein_no]
    seq1_n = data['allSeq']['Sequence'][protein_no]
    str1 = data['allSeq']['Structure'][protein_no]
    
    win = hankel(seq1_n[:W],seq1_n[W-1:])
    j1=kron(win,np.ones((20,1))).toarray()
    j2 = kron(np.ones(win.shape),np.outer(np.array(np.arange(1,21)),np.ones(1))).toarray()
    input_seq = list(j1==j2)
    X_temp.append(input_seq)
    
    j3 = kron(np.ones((1,win.shape[1])),np.outer(np.array([ord(x) for x in list('CEH')]),np.ones(1))).toarray()
    j4 = kron(np.array([ord(x) for x in list(str1[int((W-1)/2):-int((W+1)/2-1)])]),np.ones((3,1))).toarray()
    input_struc = list(j4==j3)
    y_temp.append(input_struc)

X_data = np.transpose(np.hstack(X_temp)).astype(np.float32)
y_data = np.transpose(np.hstack(y_temp)).astype(np.float32)
y_labels = np.argmax(y_data,axis=1)
print ("Length of input data : {}".format(X_data.shape))
print ("Length of input target : {}".format(y_data.shape))


#The data is split into training and testing datasets
X_train_data, X_test_data,y_train_data,y_test_data = train_test_split(X_data,y_data,test_size=0.33)
print ("Length of Training data = {}".format(X_train_data.shape))
print ("Length of Test data = {}".format(X_test_data.shape))

Length of input data : (15471, 340)
Length of input target : (15471, 3)
Length of Training data = (10365, 340)
Length of Test data = (5106, 340)


## Neural Network 

In [61]:
# Parameters to using during training.
learning_rate = 0.01
training_epochs = 100
batch_size = 125
display_step = 20
total_batch = int(X_train_data.shape[0]/batch_size)


#Creating placeholder to feed the neural network
x = tf.placeholder(tf.float32,shape=[None,X_data.shape[1]])
y = tf.placeholder(tf.float32,shape=[None,3])

#Parameters of the input layer, hidden layers and output layer
n_input = X_data.shape[1]
n_classes = 3
layers = [5]

#Function to generate the neural network based on the Paramters
def generate_model(n_input=n_input,n_classes=n_classes,layers=layers):
    dimension_1 = [n_input]+layers
    dimension_2 = layers+[n_classes]


    names = ['h'+str(i+1) for i in range(len(layers))]+['out']

    weights_var = [tf.Variable(tf.random_normal([i,j])) for i,j in zip(dimension_1,dimension_2)]
    biases_var = [tf.Variable(tf.random_normal([i])) for i in (dimension_2)]

    weights_1 = dict(zip(names,weights_var))
    biases_1 = dict(zip(names,biases_var))

    layer = x

    for key in sorted(weights_1):
        print (weights_1[key])
        layer = tf.add(tf.matmul(layer,weights_1[key]),biases_1[key])
        #if key != 'out':
            #layer = tf.nn.relu(layer) 
    
    return layer



In [62]:
# Function to generate batches for input during the training
def get_batch(x,y):
    N =x.shape[0]
    idx = np.random.choice(N,size=batch_size)
    x_batch = x[idx]
    y_batch = y[idx]
    return x_batch, y_batch

#### Creating the model and defining the loss and optimizer

In [63]:
model = generate_model(layers=[20,10])# deep_net(x, weights, biases)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(model,y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

init = tf.global_variables_initializer()

Tensor("Variable_110/read:0", shape=(340, 20), dtype=float32)
Tensor("Variable_111/read:0", shape=(20, 10), dtype=float32)
Tensor("Variable_112/read:0", shape=(10, 3), dtype=float32)


### Training Neural network and predicition

In [64]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        
        avg_loss = 0.0
        for i in range (total_batch):
            batch_x, batch_y = get_batch(X_train_data,y_train_data)
            _,l = sess.run([optimizer,loss],feed_dict={x:batch_x,y:batch_y})
            avg_loss += l/total_batch
        
        if epoch%display_step == 0:
            print ("Epoch: {0}, cost : {1}".format(epoch,avg_loss))
    print ("Optimization Finished!")
    #x_test, y_test = get_batch(X_data,y_data)
    
    x_test, y_test = X_test_data, y_test_data
    
    correct_prediction = tf.equal(tf.argmax(model,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({x: x_test, y: y_test}))  
    
    y_pred = sess.run(tf.arg_max(model,1),feed_dict={x: x_test, y: y_test})
    y_true = np.argmax(y_test,1)
    
    cm = confusion_matrix(y_true=y_true,y_pred=y_pred)
    print ("\nConfusion matrix : \n{}".format(cm))
    
    print ("\nPrecision score : {}".format(precision_score(y_true,y_pred,average='weighted')))
    print ("\nRecall score : {}".format(recall_score(y_true,y_pred,average='weighted')))
    print ("\nf1 score : {}".format(f1_score(y_true,y_pred,average='weighted')))
    
    c_report = classification_report(y_true,y_pred)
    print ("\n Classification report : \n{}".format(c_report))
        

Epoch: 0, cost : 12.434651642310913
Epoch: 20, cost : 0.885101211507146
Epoch: 40, cost : 0.8614070524529716
Epoch: 60, cost : 0.838609508624891
Epoch: 80, cost : 0.8277248427635284
Optimization Finished!
Accuracy: 0.599882

Confusion matrix : 
[[1762  273  348]
 [ 377  491  190]
 [ 557  298  810]]

Precision score : 0.5967623021778418

Recall score : 0.599882491186839

f1 score : 0.5951250973021804

 Classification report : 
             precision    recall  f1-score   support

          0       0.65      0.74      0.69      2383
          1       0.46      0.46      0.46      1058
          2       0.60      0.49      0.54      1665

avg / total       0.60      0.60      0.60      5106



#### The accuracy is ~60%. Currently, there are several advanced algorithms that have achieved higher prediction accuracy taking not only the amino acid sequence, but also comparing to other similar proteins in the protein database as input. 